#Product Recommendation

## importing required libraries and creating sythetic data

In [ ]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import random
from faker import Faker
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [ ]:

# Initialize Faker
fake = Faker()

def generate_interaction_data(n_customers, n_products, n_interactions):
    data = []
    customer_ids = [fake.uuid4() for _ in range(n_customers)]
    product_ids = [fake.uuid4() for _ in range(n_products)]

    for _ in range(n_interactions):
        customer_id = random.choice(customer_ids)
        product_id = random.choice(product_ids)
        interaction_type = random.choice(["purchased", "viewed", "clicked"])
        interaction_date = fake.date_this_year()

        data.append({
            "customer_id": customer_id,
            "product_id": product_id,
            "interaction_type": interaction_type,
            "interaction_date": interaction_date
        })

    return pd.DataFrame(data)

In [ ]:
# Parameters
n_customers = 500  # Number of unique customers
n_products = 100    # Number of unique products
n_interactions = 10000  # Total number of interactions

# Generate the dataset
interaction_df = generate_interaction_data(n_customers, n_products, n_interactions)

In [ ]:
interaction_df.head()

,customer_id,product_id,interaction_type,interaction_date
0,cb43c7ac-bf5c-4330-b301-9967ef5b9a7b,feb37854-b711-4356-bea6-74e898c6a341,purchased,2025-01-18
1,63b6f65c-8a8b-4885-8056-512f66673cce,2e9f16fe-59c8-4154-ba7b-830a63fbbf57,clicked,2025-01-31
2,d13030f3-9375-431d-a367-78ccda43d9f8,1fddf710-7739-4972-b44f-cc1383aaf25c,viewed,2025-01-20
3,7cb060ee-82d7-4934-90c7-465151f7c4e1,399f9180-5c72-446a-a2ba-666ece0614a1,purchased,2025-03-19
4,36c2eb04-6b3e-4f72-8097-c50e0436c18c,70391e02-b4db-404e-b8ee-e3d2b93789ae,clicked,2025-01-19


## Data preprocessing

In [ ]:
# Assign numerical values to interaction types
interaction_mapping = {'viewed': 1, 'clicked': 2, 'purchased': 3}
interaction_df['interaction_value'] = interaction_df['interaction_type'].map(interaction_mapping)

# Create user-product interaction matrix
interaction_matrix = interaction_df.pivot_table(index='customer_id', columns='product_id', values='interaction_value', fill_value=0)

In [ ]:
interaction_matrix

product_id,00fa96bf-c370-4d5a-abc8-1bc8e3e68057,02a874fd-0cd8-4b93-bb5b-fcc3e2a3e596,0a579637-d327-4c14-95fa-8dc3c62055fa,0ba2170c-9cab-40a8-88da-3c869fe92e55,0c32a379-1b98-461e-b27f-7c756168550a,0f62ec86-ca1b-4811-9990-d44f4b3b1bb3,0fd0f634-050b-4663-a5ba-11699088801f,12a887e7-83d3-4243-84a3-c4298f9debf0,15464f81-388e-42d2-abcf-c59772fc05cd,1592dc08-0b43-4571-a61a-9d2886a35fd4,...,ebf7fb66-29a0-4cb0-b14b-e49d52735083,ec5f2c0a-72f1-455b-8425-8345c7d4e10d,eef86f1b-2769-4a2f-991f-392a894361d3,f72bb812-3cc3-4a01-9f64-ef83feb4429f,fa66959c-34c4-4b5b-858f-b818f1e6f8c9,fb900498-bcf0-4757-9f17-f72e14b3bfd8,fbb761a4-23b5-401b-b684-a6dc6487cafa,fd49c5ca-fa17-478a-b9e3-d9b566474d7f,fe0ae9bb-d7dd-4934-9e6d-042246f74574,feb37854-b711-4356-bea6-74e898c6a341
customer_id,,,,,,,,,,,,,,,,,,,,,
00e11cdb-5918-46bb-ba06-d1f108906a84,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00fba559-6b01-4cc5-a440-375c5d5e1d4b,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,2.0
01cb1bdd-fb8c-47d9-86fe-9277fcaacc94,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0
0247efd4-58a6-405f-b5f1-1a4f46b03ad6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,3.0,1.0
0254c1b6-f4ba-419b-b4c8-7c79a9bbd778,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fc35459a-a200-4e62-b14c-3c9b87a2bbf8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fd9e971a-6836-4b8f-8b9f-5df4017a7649,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
ff03c0b6-16a7-4df1-9ea8-d7f1bb41d269,3.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Interaction Matrix and Cosine Similarity

In [ ]:
# Convert to sparse matrix for efficiency
sparse_matrix = csr_matrix(interaction_matrix.values)

# Compute cosine similarity between items
item_similarity = cosine_similarity(sparse_matrix.T)

# Convert similarity matrix to DataFrame
item_similarity_df = pd.DataFrame(item_similarity, index=interaction_matrix.columns, columns=interaction_matrix.columns)

## Get recommendation

In [ ]:
# Function to get item recommendations
def get_item_recommendations(item_id, num_recommendations):
    similar_items = item_similarity_df[item_id].sort_values(ascending=False).iloc[1:num_recommendations+1]
    return similar_items.index.tolist()

In [ ]:
# Example: Get recommendations for a specific
user_product = '2e9f16fe-59c8-4154-ba7b-830a63fbbf57'
recommended_items = get_item_recommendations(user_product, 5)
print(f"Items similar to {user_product}: ","\n\t","\n\t".join(recommended_items))

Items similar to 2e9f16fe-59c8-4154-ba7b-830a63fbbf57:  
	 2cf2c327-3a5d-449e-90f5-8b9ae91e0125
	f72bb812-3cc3-4a01-9f64-ef83feb4429f
	2c01fccf-30e6-4960-8673-87a18383723e
	ad1c779c-3ad7-47b0-bc8b-89e6b1edc46f
	bed983ea-29b1-4b8c-89eb-5854df763315
